In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import re
from ipyleaflet import Map, Marker, Popup, basemaps, basemap_to_tiles
from ipywidgets import HTML
from os import listdir
from os.path import join
import datetime
from functools import reduce

In [3]:
adcp_locations = pd.read_csv(r"C:\Users\PAKHUIYING\OneDrive - Nanyang Technological University\NTU\SMI\DHI data\2. ADCP Monitoring Data\2. ADCP Monitoring Data\ADCP_locations.csv",engine='python')
adcp_locations

,Sensor ID,Owner,Type,Latitude,Longitude,Deployment Date
0,ADCP1,MPA TTP2,Acoustic,1° 12' 42.012'' N,103° 38' 40.992'' E,1-Jun-18
1,ADCP2,MPA TTP2,Acoustic,1° 13' 24.316'' N,103° 40' 15.578'' E,27-Feb-19
2,ADCP7,MPA TTP2,Acoustic,1° 15' 12.45'' N,103° 39' 33.21'' E,23-Feb-19
3,ADCP8,MPA TTP2,Acoustic,"1° 14' 4.83"" N","103° 40' 6.27"" E",27-Feb-20


In [4]:
def convert_dms_to_degree(coord):
    coord_split =  re.split('[°|\'|\"]',coord.replace(' ',''))
    deg, minutes, seconds, direction = [i for i in coord_split if i != '']
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

In [5]:
adcp_locations['Latitude_degree'] = adcp_locations.Latitude.apply(convert_dms_to_degree)
adcp_locations['Longitude_degree'] = adcp_locations.Longitude.apply(convert_dms_to_degree)

In [33]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=tuple(adcp_locations.iloc[-1,-2:].values),
    zoom=13
    )
m

for index,row in adcp_locations.iterrows():
    coord = (row['Latitude_degree'],row['Longitude_degree'])
    marker = Marker(location=coord)
    m.add_layer(marker)
    msg = HTML()
    msg.value = row['Sensor ID']
    marker.popup = msg

m

Map(center=[1.234675, 103.66840833333333], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

# Import current data

In [41]:
adcp_data_directory = r"C:\Users\PAKHUIYING\OneDrive - Nanyang Technological University\NTU\SMI\DHI data\2. ADCP Monitoring Data\2. ADCP Monitoring Data\ADCP1_ADCP2_ADCP7_ADCP8_202010-202012"
adcp_data_names = [f.split('_')[1] for f in listdir(adcp_data_directory)]
adcp_data_names = list(set(adcp_data_names))
adcp_data_param = ['_'.join(f.split('_')[2:]) for f in listdir(adcp_data_directory)]
adcp_data_param = list(set(adcp_data_param))
# adcp_data_units = [f.split('_')[3] for f in listdir(adcp_data_directory)]
# adcp_data_units = list(set(adcp_data_units))
print(adcp_data_names,adcp_data_param)

['ADCP7', 'ADCP8', 'ADCP2', 'ADCP1'] ['Direction_(deg).csv', 'Speed_(m_s).csv', 'Depth_(m).csv']


In [58]:
adcp_dict = {n:{p.replace('.csv',''):None for p in adcp_data_param} for n in adcp_data_names}
for names in adcp_data_names:
    for param in adcp_data_param:
        df = pd.read_csv(join(adcp_data_directory,'_'.join(("TTP2",names,param))))
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        df = df.iloc[:,:-1]
        p = param.replace('.csv','')
        adcp_dict[names][p] = df

In [63]:
for name, params in adcp_dict.items():
    df_list = []
    for p, df in params.items():
        df_list.append(df)
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Datetime'],
                                            how='inner'), df_list)
    df_merged.columns = ['Datetime'] + list(params)
    df_merged.to_csv(join(adcp_data_directory,'{}_merged_data.csv'.format(name)),index=False)
df_merged

,Datetime,Direction_(deg),Speed_(m_s),Depth_(m)
0,2020-10-01 06:00:00,160.32,0.040,20.5
1,2020-10-01 06:05:00,161.72,0.040,20.5
2,2020-10-01 06:10:00,185.70,0.059,20.5
3,2020-10-01 06:15:00,178.32,0.063,20.5
4,2020-10-01 06:20:00,181.87,0.041,20.5
...,...,...,...,...
26371,2021-01-01 05:40:00,234.25,0.252,21.4
26372,2021-01-01 05:45:00,250.07,0.184,21.4
26373,2021-01-01 05:50:00,252.85,0.163,21.4
26374,2021-01-01 05:55:00,206.72,0.174,21.3


# Load merged data

In [3]:
adcp_data_directory = r"C:\Users\PAKHUIYING\OneDrive - Nanyang Technological University\NTU\SMI\DHI data\2. ADCP Monitoring Data\2. ADCP Monitoring Data\ADCP1_ADCP2_ADCP7_ADCP8_202010-202012"
df_merged_list = {f[:5]: pd.read_csv(join(adcp_data_directory,f)) for f in listdir(adcp_data_directory) if 'merged_data' in f}
len(df_merged_list)

4

In [4]:
mean_values = {k: None for k in list(df_merged_list)}
for names, df in df_merged_list.items():
    df_copy = df.iloc[:,1:]
    row_names = df_copy.columns
    mean_values[names] = df_copy.apply(np.mean).values
avg_df = pd.DataFrame.from_dict(mean_values,orient='index',columns = row_names)
avg_df

,Direction_(deg),Speed_(m_s),Depth_(m)
ADCP1,193.218712,0.281122,21.798157
ADCP2,169.513373,0.222205,17.508469
ADCP7,207.170448,0.051649,10.650789
ADCP8,214.745239,0.069687,14.837948


In [5]:
import matplotlib.pyplot as plt
import pandas as pd
avg_df.plot.bar()

<AxesSubplot:>

: 

: 

# Geopandas

In [5]:
gdf = gpd.GeoDataFrame(adcp_locations, geometry=gpd.points_from_xy(adcp_locations.Longitude_degree, adcp_locations.Latitude_degree))
gdf

,Sensor ID,Owner,Type,Latitude,Longitude,Deployment Date,Latitude_degree,Longitude_degree,geometry
0,ADCP1,MPA TTP2,Acoustic,1° 12' 42.012'' N,103° 38' 40.992'' E,1-Jun-18,1.211670,103.644720,POINT (103.64472 1.21167)
1,ADCP2,MPA TTP2,Acoustic,1° 13' 24.316'' N,103° 40' 15.578'' E,27-Feb-19,1.223421,103.670994,POINT (103.67099 1.22342)
2,ADCP7,MPA TTP2,Acoustic,1° 15' 12.45'' N,103° 39' 33.21'' E,23-Feb-19,1.253458,103.659225,POINT (103.65923 1.25346)
3,ADCP8,MPA TTP2,Acoustic,"1° 14' 4.83"" N","103° 40' 6.27"" E",27-Feb-20,1.234675,103.668408,POINT (103.66841 1.23467)


In [7]:
shp_fp = r"C:\Users\PAKHUIYING\OneDrive - Nanyang Technological University\NTU\SMI\DHI data\master-plan-2014-region-boundary-web\master-plan-2014-region-boundary-web-shp\MP14_REGION_WEB_PL.shp"
sg_boundary = gpd.read_file(shp_fp)
sg_boundary

,OBJECTID,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,CENTRAL REGION,CR,F6D4903B6C0B72F8,2014-12-05,27836.5573,31929.9186,131065.464453,1.364056e+08,"MULTIPOLYGON (((29717.297 38342.160, 29772.191..."
1,2,EAST REGION,ER,646F1B60DAFE0282,2014-12-05,42245.8477,36760.4998,60375.722057,1.129697e+08,"MULTIPOLYGON (((41343.113 40442.099, 41375.108..."
2,3,NORTH REGION,NR,3B51FFC82787B16B,2014-12-05,23647.9008,44176.7871,87426.777164,1.394056e+08,"MULTIPOLYGON (((21693.062 48119.514, 21702.562..."
3,4,NORTH-EAST REGION,NER,21277DA34DEE03B8,2014-12-05,41561.9228,41689.7948,106583.903458,1.360536e+08,"MULTIPOLYGON (((35103.373 44091.162, 35133.011..."
4,5,WEST REGION,WR,11540153B663CA9B,2014-12-05,12896.4360,33986.5714,258264.026231,2.571103e+08,"MULTIPOLYGON (((12861.383 32207.492, 12860.555..."


In [9]:
sg_boundary[sg_boundary.REGION_N == 'WEST REGION']
type(sg_boundary)

geopandas.geodataframe.GeoDataFrame

In [10]:
ax = sg_boundary[sg_boundary.REGION_N == 'WEST REGION'].plot(color='white', edgecolor='black')

: 

: 

In [3]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world
# We restrict to South America.
ax = world[world.continent == 'South America'].plot(color='white', edgecolor='black')

: 

: 